In [71]:
from PIL import Image
import pylab as pl
from scipy import ndimage
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import cv2
import math
import serial
import time
from PIL import Image
from scipy import ndimage
import scipy.ndimage
from matplotlib.patches import Polygon

In [72]:
border_point=np.array([[10,10],[20,500],[320,450],[400,20]])
circle_center=np.ones(2)
circle_center[0]=int(np.sum(border_point[:,0])/(np.size(border_point[:,0])+0.001))
circle_center[1]=int(np.sum(border_point[:,1])/(np.size(border_point[:,1])+0.001))
# 首先对于获取的边界点，我们先分别生成正方形和圆的坐标
# 然后我们直接使用M矩阵将坐标变换成想要的坐标
# 其中，两个坐标将是分别生成的，以保留顺序
gray=np.zeros((480,640))
ad=cv2.imread('ad.png')
square_origin=cv2.imread('square_origin.png')
circle_origin=cv2.imread('circle_origin.png')
# 原先的点
pts = np.float32([[0,0],[0,square_origin.shape[1]],[square_origin.shape[0],square_origin.shape[1]],[square_origin.shape[0],0]])
# # 后面的点
border_point=np.array([[10,10],[20,500],[320,450],[400,20]])
# 由于变换的特殊性，这里后面的点是列行，反着来的
border_point_1=np.zeros(np.shape(border_point))
border_point_1[:,0]=border_point[:,1]
border_point_1[:,1]=border_point[:,0]
pts1 = np.float32(border_point_1)
# 生成最为关键的变换矩阵
M = cv2.getPerspectiveTransform(pts,pts1)
# 变换正方形
square_trans = cv2.warpPerspective(square_origin,M,(ad.shape[1],ad.shape[0]))
square_trans = cv2.cvtColor(square_trans, cv2.COLOR_RGB2GRAY)
_, square_trans = cv2.threshold(square_trans, 127, 255, cv2.THRESH_BINARY)
# 变换圆
circle_trans = cv2.warpPerspective(circle_origin,M,(ad.shape[1],ad.shape[0]))
circle_trans = cv2.cvtColor(circle_trans, cv2.COLOR_RGB2GRAY)
_, circle_trans = cv2.threshold(circle_trans, 127, 255, cv2.THRESH_BINARY)

In [73]:
# 然后获取每条直线的斜率（方向）,范围是0到2*pi
k_angle = np.array(range(0, int(200 * (2 * math.pi)), 5)) / 200 - math.pi
# 获取边框上的点，用于之后遍历路径
stripe = np.array(np.where(square_trans))
stripe_center = circle_center
# 获取整个图面的中心点
# 所有的点都是以行列的形式标注的，先取行，再取列
square_trans[int(stripe_center[0]), int(stripe_center[1])] = 255
# 接下来找出每个有值的点所具有对应三角值
# 找出相对位置
k_re = stripe[0, :] - stripe_center[0], stripe[1, :] - stripe_center[1]
k_re = np.array(k_re)
k_stripe_pi = np.arctan2(k_re[0, :], k_re[1, :])
# 找出所有的斜率对应的最接近的点
# 将找出的边界点存入对应的road_point矩阵中,最后保存为square_point
# road用于显示最后的路径图像
# road_point收集有顺序的点坐标
road=np.zeros(np.shape(gray))
road_point=np.ones((np.size(k_angle),2))
for k in range(0, np.size(k_angle)):
    k_rela = np.abs(k_stripe_pi - k_angle[k])
    min_index = np.argmin(k_rela)
    # road[int(stripe[0, min_index]), int(stripe[1, min_index])] = 255
    road_point[k,:]=[int(stripe[0, min_index]), int(stripe[1, min_index])]
square_point_num=np.shape(road_point)[0]
# 调整正方形的顺序
square_point=np.zeros(np.shape(road_point))
square_point[0:int(square_point_num/8*7),:]=road_point[(int(square_point_num)-int(square_point_num/8*7))-1:square_point_num-1,:]
square_point[int(square_point_num/8*7)-1:int(square_point_num)-1,:]=road_point[0:int(square_point_num-int(square_point_num/8*7)),:]

In [74]:
# 获取边框上的点，用于之后遍历路径
stripe = np.array(np.where(circle_trans))
# 获取整个图面的中心点
# 所有的点都是以行列的形式标注的，先取行，再取列
circle_trans[int(stripe_center[0]), int(stripe_center[1])] = 255
# 接下来找出每个有值的点所具有对应三角值
# 找出相对位置
k_re = stripe[0, :] - stripe_center[0], stripe[1, :] - stripe_center[1]
k_re = np.array(k_re)
k_stripe_pi = np.arctan2(k_re[0, :], k_re[1, :])
# 找出所有的斜率对应的最接近的点
# 将找出的边界点存入对应的road_point矩阵中,最后保存为square_point
# road用于显示最后的路径图像
# road_point收集有顺序的点坐标
# road=np.zeros(np.shape(gray))
road_point=np.ones((np.size(k_angle),2))
for k in range(0, np.size(k_angle)):
    k_rela = np.abs(k_stripe_pi - k_angle[k])
    min_index = np.argmin(k_rela)
    # road[int(stripe[0, min_index]), int(stripe[1, min_index])] = 255
    road_point[k,:]=[int(stripe[0, min_index]), int(stripe[1, min_index])]
circle_point_num=np.shape(road_point)[0]
# 调整圆的顺序
circle_point=np.zeros(np.shape(road_point))
circle_point[0:int(circle_point_num/8*7),:]=road_point[(int(circle_point_num)-int(circle_point_num/8*7))-1:circle_point_num-1,:]
circle_point[int(circle_point_num/8*7)-1:int(circle_point_num)-1,:]=road_point[0:int(circle_point_num-int(circle_point_num/8*7)),:]